In [ ]:
# Data gathering script
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from urllib.parse import urljoin

BASE_URL = "https://www.insd.bf/"
# https://www.insd.bf/sites/default/files/2024-08/ADAPTATION_DES_METHODOLOGIES_STATISTIQUES.pdf
# https://burkinafaso.opendataforafrica.org/
# Exemple de points d'entrée; adaptez selon les sections pertinentes du site
ENTRY_POINTS = [
    "fr/definitions-concept",       # ajustez en fonction de l’URL réelle
    "fr/methodes",
    "fr/resultats",
    "",
]

HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; DataScraper/1.0; +https://example.org/bot)",
    "Accept-Language": "fr-FR,fr;q=0.9",
}

def fetch_page(url):
    try:
        resp = requests.get(url, headers=HEADERS, timeout=15)
        resp.raise_for_status()
        return resp.text
    except requests.RequestException as e:
        print(f"Échec du chargement {url}: {e}")
        return None

def parse_indicators(soup):
    data = []
    # Exemple: trouver les tableaux ou sections avec données numériques
    # Ajustez les sélecteurs en fonction de la structure réelle du site INS D
    for bloc in soup.select("table, .indicateurs, .data-block, table.data"):
        # Si c'est un tableau, essayer d'en tirer des paires label/value
        rows = bloc.select("tr")
        for r in rows:
            cells = r.find_all(["td","th"])
            if len(cells) >= 2:
                label = cells[0].get_text(strip=True)
                value = cells[1].get_text(strip=True)
                if label and value:
                    data.append({"label": label, "value": value})
        if data:
            break
    return data

def extract_from_entry(entry_url):
    html = fetch_page(entry_url)
    if not html:
        return []
    soup = BeautifulSoup(html, "html.parser")
    indicators = parse_indicators(soup)
    return indicators

def main():
    all_records = []
    for path in ENTRY_POINTS:
        url = urljoin(BASE_URL, path if path.startswith("/") is False else path)
        html = fetch_page(url)
        if not html:
            continue
        soup = BeautifulSoup(html, "html.parser")

        # Exemple de navigation: trouver toutes les sections nécessitant des pages supplémentaires
        # Si le site a des liens vers des pages détaillées, collectez-les ici
        links = []
        for a in soup.select("a[href]"):
            href = a['href']
            if href and ("/publication/" in href or "/statistique/" in href or "Rapport" in a.get_text()):
                full = urljoin(BASE_URL, href)
                links.append(full)

        # Si pas de liens spécifiques, tenter d'extraire directement depuis la page courante
        if not links:
            indicators = parse_indicators(soup)
            for rec in indicators:
                rec['source'] = url
            all_records.extend(indicators)
        else:
            # Parcourir les pages liées
            seen = set()
            for link in links:
                if link in seen:
                    continue
                seen.add(link)
                time.sleep(1.0)  # respect du serveur
                page_html = fetch_page(link)
                if not page_html:
                    continue
                page_soup = BeautifulSoup(page_html, "html.parser")
                indicators = parse_indicators(page_soup)
                for rec in indicators:
                    rec['source'] = link
                all_records.extend(indicators)

        time.sleep(1.0)  # delay entre les entrées pour limiter la charge serveur

    # Nettoyage et export
    if all_records:
        df = pd.DataFrame(all_records)
        # Si les données ont des colonnes « label » et « value », on peut les pivotER
        if set(["label","value","source"]).issubset(df.columns):
            # Optionnel: pivoter les données par label si nécessaire
            df_pivot = df.pivot_table(index=None, columns="label", values="value", aggfunc="first")
            df_pivot.to_csv("insd_burkina_faso_indicateurs.csv", index=False)
        else:
            df.to_csv("insd_burkina_faso_indicateurs.csv", index=False)
        print("Export terminé: insd_burkina_faso_indicateurs.csv")
    else:
        print("Aucune donnée extraites.")

if __name__ == "__main__":
    main()


Export terminé: insd_burkina_faso_indicateurs.csv
